In [ ]:
import pymongo
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["SA"]
collection = db["PRJ-16"]

In [ ]:
# # Count the total number of documents
# total_documents = collection.count_documents({})
# total_documents

In [ ]:
from bson.objectid import ObjectId
from datetime import datetime, timedelta


def get_raw_documents_for_date(collection, date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d")
    end_date = date + timedelta(hours=32)

    oid_start = ObjectId.from_datetime(date)
    oid_end = ObjectId.from_datetime(end_date)

    cursor = collection.find({
        "_id": {"$gte": oid_start, "$lt": oid_end}
    })
    return list(cursor)

In [ ]:
import pandas as pd


def process_sensors_from_raw(raw_docs, sensors, minutes):
    records = []
    for doc in raw_docs:
        time = doc['_id'].generation_time.replace(tzinfo=None)
        for sensor in sensors:
            if sensor in doc and isinstance(doc[sensor], dict):
                value_field = 't' if sensor.startswith('T') else 's'
                value = doc[sensor].get(value_field)
                if value is not None:
                    records.append({
                        'time': time,
                        'sensor': sensor,
                        'value': value
                    })

    df = pd.DataFrame(records)
    if df.empty:
        return pd.DataFrame()  # early exit

    # Round to time bins
    df['time_bin'] = df['time'].dt.floor(f'{minutes}min')

    # Pivot to get medians per sensor per bin
    df_grouped = df.groupby(['time_bin', 'sensor'])['value'].median().unstack()
    df_grouped.reset_index(inplace=True)
    return df_grouped.rename(columns={'time_bin': 'time'})

In [ ]:
raw_docs = get_raw_documents_for_date(collection, "2025-05-31")

df_sensors = process_sensors_from_raw(
    raw_docs=raw_docs,
    sensors=[
        'T6', 'T5', 'T4', 'T3', 'T2', 'T1',
        "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
        "S15", "S16", "S17", "S18", "S19", "S20",
        "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29"
    ],
    minutes=5
)

In [ ]:
def tare_s_sensors_by_nearest_6am(df_sensors):
    s_sensor_columns = [
        col for col in df_sensors.columns if col.startswith('S')]
    if df_sensors.empty or not s_sensor_columns:
        return df_sensors  # early return for empty data

    # Compute 6:00 AM reference for each row
    target_time = pd.Timedelta(hours=6)
    df_sensors['time_diff'] = abs(
        df_sensors['time'] - (df_sensors['time'].dt.normalize() + target_time))

    # Find row index closest to 6:00 AM
    nearest_time_index = df_sensors['time_diff'].idxmin()

    # Tare all S sensors using values at that time
    for s_sensor in s_sensor_columns:
        tare_value = df_sensors.at[nearest_time_index, s_sensor]
        if pd.notnull(tare_value):
            df_sensors[s_sensor] -= tare_value

    return df_sensors.drop(columns=['time_diff'])


def tare_and_multiply_s_sensors(df_sensors, s_multiplier=3.11e-9 * 210e3):
    s_sensor_columns = [
        col for col in df_sensors.columns if col.startswith('S')]
    if df_sensors.empty or not s_sensor_columns:
        return df_sensors  # early return for empty data

    for s_sensor in s_sensor_columns:
        df_sensors[s_sensor] = df_sensors[s_sensor] * s_multiplier

    return df_sensors

In [ ]:
# Assuming df_sensors is your DataFrame created by `create_sensor_dataframe`
df_sensors = tare_s_sensors_by_nearest_6am(
    df_sensors)  # Tare based on the nearest 6 AM value
# Multiply the tared values by s_multiplier
df_sensors = tare_and_multiply_s_sensors(df_sensors)

In [ ]:
def plot_s_and_t_sensors(df_sensors, s_sensor, t_sensor, title="Koncert"):
    fig = go.Figure()

    # Extract time data
    time_data = df_sensors['time']

    # Plot S sensor values on the left y-axis
    fig.add_trace(go.Scatter(
        x=time_data,
        y=df_sensors[s_sensor],
        mode='lines',
        name=f"{s_sensor} Napätia",
        yaxis='y1'  # Left y-axis
    ))

    # Plot T sensor values on the right y-axis
    fig.add_trace(go.Scatter(
        x=time_data,
        y=df_sensors[t_sensor],
        mode='lines',
        name=f"{t_sensor} Teploty",
        yaxis='y2'  # Right y-axis
    ))

    # Update layout to add secondary y-axis and format the plot
    fig.update_layout(
        title=title,
        # xaxis_title="Time",
        yaxis=dict(
            title=f"{s_sensor} Napätia [MPa]",  # Left y-axis title
            side="left"
        ),
        yaxis2=dict(
            title=f"{t_sensor} Teploty [°C]",  # Right y-axis title
            overlaying='y',  # Overlay on the same plot
            side='right',
            showgrid=False  # Turn off grid for the temperature axis
        ),
        legend=dict(
            orientation="h",  # Horizontal legend
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template="plotly"
    )

    # Show the plot
    fig.show()

In [ ]:
def plot_multiple_s_and_t_sensors(
    df_sensors,
    s_sensors,
    t_sensors,
    title="Koncert - viac senzorov",
    s_section_map=None,
    t_label_map=None,
    s_color_map=None,
    t_color_map=None
):
    fig = go.Figure()
    time_data = df_sensors['time']

    # Add strain sensors (S) on y1
    for s_sensor in s_sensors:
        if s_sensor in df_sensors.columns:
            section = s_section_map.get(s_sensor, "") if s_section_map else ""
            label = f"{s_sensor} ({section})" if section else s_sensor
            color = s_color_map.get(
                section, None) if s_color_map and section else None

            fig.add_trace(go.Scatter(
                x=time_data,
                y=df_sensors[s_sensor],
                mode='lines',
                name=label,
                yaxis='y1',
                line=dict(dash='solid', color=color)
            ))

    # Add temperature sensors (T) on y2 with dashed lines
    for t_sensor in t_sensors:
        if t_sensor in df_sensors.columns:
            label = t_label_map.get(
                t_sensor, f"{t_sensor} Teplota") if t_label_map else f"{t_sensor} Teplota"
            color = t_color_map.get(t_sensor, None) if t_color_map else None

            fig.add_trace(go.Scatter(
                x=time_data,
                y=df_sensors[t_sensor],
                mode='lines',
                name=label,
                yaxis='y2',
                line=dict(dash='dash', color=color)
            ))

    fig.update_layout(
        title=title,
        yaxis=dict(title="Napätia [MPa]", side="left"),
        yaxis2=dict(title="Teploty [°C]", overlaying='y',
                    side='right', showgrid=False),
        legend=dict(orientation="h", yanchor="bottom",
                    y=1.02, xanchor="right", x=1),
        template="plotly_white"
    )

    fig.show()

#### 7. Finále 29. apríl 2025

In [ ]:
# Set date and match label only once
date = "2025-05-31"
match_label = "Koncert - Rytmus"

# Pull raw documents for the selected date
raw_docs = get_raw_documents_for_date(collection, date)

# Convert to DataFrame (5-minute bins)
df_sensors = process_sensors_from_raw(
    raw_docs=raw_docs,
    sensors=[
        'T6', 'T5', 'T4', 'T3', 'T2', 'T1',
        "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
        "S15", "S16", "S17", "S18", "S19", "S20",
        "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29"
    ],
    minutes=10
)

# Tare + multiply S sensors
df_sensors = tare_s_sensors_by_nearest_6am(df_sensors)
df_sensors = tare_and_multiply_s_sensors(df_sensors)

In [ ]:
df_sensors.columns

In [ ]:
# Format date in Slovak: "29. apríl 2025"
sk_months = {
    1: "január", 2: "február", 3: "marec", 4: "apríl", 5: "máj", 6: "jún",
    7: "júl", 8: "august", 9: "september", 10: "október", 11: "november", 12: "december"
}
dt = pd.to_datetime(date)
date_label = f"{dt.day}. {sk_months[dt.month]} {dt.year}"

# Plot with automatic title
plot_s_and_t_sensors(
    df_sensors, s_sensor='S27', t_sensor='T4',
    title=f"Dolná tribúna (S27), {date_label} – {match_label} (UTC čas)"
)

plot_s_and_t_sensors(
    df_sensors, s_sensor='S29', t_sensor='T6',
    title=f"Horná tribúna (S29), {date_label} – {match_label} (UTC čas)"
)

plot_s_and_t_sensors(
    df_sensors, s_sensor='S28', t_sensor='T5',
    title=f"Stĺp (S28), {date_label} – {match_label} (UTC čas)"
)

# REZ 1 - Sever oblúka
plot_s_and_t_sensors(
    df_sensors, s_sensor='S7', t_sensor='T1',
    title=f"Sever oblúka R1, Horná pásnica (S7), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S10', t_sensor='T1',
    title=f"Sever oblúka R1, Spodná pásnica (ľavá) (S10), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S9', t_sensor='T1',
    title=f"Sever oblúka R1, Spodná pásnica (pravá) (S9), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S8', t_sensor='T1',
    title=f"Sever oblúka R1, Diagonála (S8), – {match_label} (UTC čas)"
)

# REZ 2 - Sever oblúka
plot_s_and_t_sensors(
    df_sensors, s_sensor='S13', t_sensor='T1',
    title=f"Sever oblúka R2, Horná pásnica (S13), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S14', t_sensor='T1',
    title=f"Sever oblúka R2, Spodná pásnica (ľavá) (S14), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S12', t_sensor='T1',
    title=f"Sever oblúka R2, Spodná pásnica (pravá) (S12), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S11', t_sensor='T1',
    title=f"Sever oblúka R2, Diagonála (S11), – {match_label} (UTC čas)"
)

## REZ 3 - STED OBLÚKA
plot_s_and_t_sensors(
    df_sensors, s_sensor='S16', t_sensor='T2',
    title=f"Stred oblúka R3, Horná pásnica (S16), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S18', t_sensor='T2',
    title=f"Stred oblúka R3, Spodná pásnica (pravá) (S18), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S15', t_sensor='T2',
    title=f"Stred oblúka R3, Diagonála (S15), – {match_label} (UTC čas)"
)

# REZ 4 - Juh oblúka
plot_s_and_t_sensors(
    df_sensors, s_sensor='S25', t_sensor='T3',
    title=f"Juh oblúka R4, Horná pásnica (S25), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S26', t_sensor='T3',
    title=f"Juh oblúka R4, Spodná pásnica (ľavá) (S26), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S23', t_sensor='T3',
    title=f"Juh oblúka R4, Spodná pásnica (pravá) (S23), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S22', t_sensor='T3',
    title=f"Juh oblúka R4, Diagonála (S22), – {match_label} (UTC čas)"
)

# REZ 5 - Juh oblúka
plot_s_and_t_sensors(
    df_sensors, s_sensor='S21', t_sensor='T3',
    title=f"Juh oblúka R5, Horná pásnica (S21), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S19', t_sensor='T3',
    title=f"Juh oblúka R5, Spodná pásnica (ľavá) (S19), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S20', t_sensor='T3',
    title=f"Juh oblúka R5, Spodná pásnica (pravá) (S20), – {match_label} (UTC čas)"
)
plot_s_and_t_sensors(
    df_sensors, s_sensor='S24', t_sensor='T3',
    title=f"Juh oblúka R5, Diagonála (S24), – {match_label} (UTC čas)"
)

In [ ]:
# Horná pásnica sensors
s_top = ["S7", "S13", "S16", "S25", "S21"]
t_top = ["T1", "T2", "T3"]

# Spodná pásnica (ľavá) sensors
s_bottom_left = ["S10", "S14", "S26", "S19"]
t_bottom_left = ["T1", "T2", "T3"]

# Spodná pásnica (pravá) sensors
s_bottom_right = ["S9", "S12", "S18", "S23", "S20"]
t_bottom_right = ["T1", "T2", "T3"]

# Diagonála sensors from REZ 1–5
s_diagonals = ["S8", "S11", "S15", "S22", "S24"]
t_diagonals = ["T1", "T2", "T3"]

s_section_map = {
    "S7": "R1",  "S13": "R2", "S16": "R3", "S25": "R4", "S21": "R5",  # horná pásnica
    "S10": "R1", "S14": "R2", "S26": "R4", "S19": "R5",              # spodná ľavá
    "S9": "R1",  "S12": "R2", "S18": "R3", "S23": "R4", "S20": "R5",  # spodná pravá
    "S8": "R1",  "S11": "R2", "S15": "R3", "S22": "R4", "S24": "R5"   # diagonály
}

t_label_map = {
    "T1": "T1 (Sever)",
    "T2": "T2 (Stred)",
    "T3": "T3 (Juh)"
}
# Colors for temperature sensors
t_color_map = {
    "T1": "#b2b2b2",
    "T2": "#808080",
    "T3": "#4d4d4d"
}

# Colors for REZ sections
s_color_map = {
    "R1": "#6670fa",
    "R2": "#f05e46",
    "R3": "#0fce9c",
    "R4": "#ae68fa",
    "R5": "#ffa561"
}

In [ ]:
plot_multiple_s_and_t_sensors(
    df_sensors,
    s_sensors=s_top,
    t_sensors=t_top,
    title=f"Horná pásnica [rezy 1~5] – {match_label} (UTC čas)",
    s_section_map=s_section_map,
    t_label_map=t_label_map,
    s_color_map=s_color_map,
    t_color_map=t_color_map
)
plot_multiple_s_and_t_sensors(
    df_sensors,
    s_sensors=s_bottom_left,
    t_sensors=list(set(t_bottom_left)),
    title=f"Spodná pásnica (ľavá) [rezy 1~5] – {match_label} (UTC čas)",
    s_section_map=s_section_map,
    t_label_map=t_label_map,
    s_color_map=s_color_map,
    t_color_map=t_color_map
)
plot_multiple_s_and_t_sensors(
    df_sensors,
    s_sensors=s_bottom_right,
    t_sensors=list(set(t_bottom_right)),
    title=f"Spodná pásnica (pravá) [rezy 1~5] – {match_label} (UTC čas)",
    s_section_map=s_section_map,
    t_label_map=t_label_map,
    s_color_map=s_color_map,
    t_color_map=t_color_map
)
plot_multiple_s_and_t_sensors(
    df_sensors,
    s_sensors=s_diagonals,
    t_sensors=list(set(t_diagonals)),  # Remove duplicates
    title=f"Diagonály [rezy 1~5] – {match_label} (UTC čas)",
    s_section_map=s_section_map,
    t_label_map=t_label_map,
    s_color_map=s_color_map,
    t_color_map=t_color_map
)

In [ ]:
sensor_labels = {
    "A30": "Oblúk sever",
    "A31": "Oblúk stred",
    "A32": "Oblúk juh",
    "A33": "Tribúna spodná",
    "A34": "Stĺp",
    "A35": "Tribúna horná"
}

sensor_colors = {
    "A30": "#16cc62",
    "A31": "#196ee6",
    "A32": "#8a27a5",
    "A33": "#c10422",
    "A34": "#19c3e6",
    "A35": "#e65d19"
}

In [ ]:
# --- SETTINGS ---
a_sensors = ["A30", "A31", "A32", "A33", "A34", "A35"]
start_date = "2025-05-31"
end_date = "2025-06-01"

# --- DATA FETCHING FUNCTION ---
def get_type_a_acceleration_magnitude(collection, sensor_id, start, end):
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d") + timedelta(days=1)
    oid_start = ObjectId.from_datetime(start_dt)
    oid_end = ObjectId.from_datetime(end_dt)

    cursor = collection.find({
        "_id": {"$gte": oid_start, "$lt": oid_end},
        f"{sensor_id}.x": {"$exists": True},
        f"{sensor_id}.fft": {"$exists": False}
    })

    records = []
    for doc in cursor:
        ts = doc["_id"].generation_time.replace(tzinfo=None)
        acc = doc.get(sensor_id, {})
        if all(k in acc for k in ("x", "y", "z")):
            x, y, z = acc["x"], acc["y"], acc["z"]
            magnitude = np.sqrt(x**2 + y**2 + z**2)
            records.append({"time": ts, "a_res": magnitude})
    return pd.DataFrame(records)

In [ ]:
for sensor in a_sensors:
    df = get_type_a_acceleration_magnitude(
        collection, sensor, start_date, end_date)
    label = sensor_labels.get(sensor, sensor)
    color = sensor_colors.get(sensor, "black")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df["time"],
        y=df["a_res"],
        mode="lines",
        line=dict(color=color, width=1),
        name=label
    ))
    fig.update_layout(
        title=f"Výsledný vektor zrýchlení – {label} – 29. apríl 2025",
        xaxis_title="Čas (UTC)",
        yaxis_title="aₙ [g]",
        template="plotly_white",
        height=500
    )
    fig.show()

In [ ]:
def get_fft_spectra_for_sensor(collection, sensor_id, start):
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = start_dt + timedelta(hours=32)

    oid_start = ObjectId.from_datetime(start_dt)
    oid_end = ObjectId.from_datetime(end_dt)

    cursor = collection.find({
        "_id": {"$gte": oid_start, "$lt": oid_end},
        f"{sensor_id}.fft": {"$exists": True}
    })

    spectrum_records = []
    for doc in cursor:
        ts = doc["_id"].generation_time.replace(tzinfo=None)
        sensor_data = doc.get(sensor_id)

        if not isinstance(sensor_data, dict):
            continue
        fft_data = sensor_data.get("fft")
        if not isinstance(fft_data, dict):
            continue
        spectrum = fft_data.get("spectrum")
        if not isinstance(spectrum, list):
            continue

        for point in spectrum:
            if isinstance(point, dict) and "f" in point and "a" in point:
                spectrum_records.append({
                    "time": ts,
                    "f": point["f"],
                    "a": point["a"]
                })

    return pd.DataFrame(spectrum_records)

In [ ]:
import matplotlib.pyplot as plt

# Define sensor IDs, colors, and labels
a_sensors = ["A30", "A31", "A32", "A33", "A34", "A35"]
sensor_colors = {
    "A30": "#16cc62",  # Oblúk sever
    "A31": "#196ee6",  # Oblúk stred
    "A32": "#8a27a5",  # Oblúk juh
    "A33": "#c10422",  # Tribúna spodná
    "A34": "#19c3e6",  # Stĺp
    "A35": "#e65d19"   # Tribúna horná
}
sensor_labels = {
    "A30": "Oblúk sever",
    "A31": "Oblúk stred",
    "A32": "Oblúk juh",
    "A33": "Tribúna spodná",
    "A34": "Stĺp",
    "A35": "Tribúna horná"
}

# Loop through each sensor
for sensor in a_sensors:
    df_fft = get_fft_spectra_for_sensor(collection, sensor, "2025-05-31")
    label = sensor_labels[sensor]
    color = sensor_colors[sensor]

    plt.figure(figsize=(9, 5))
    plt.scatter(df_fft["f"], df_fft["a"], s=10, color=color, alpha=0.7)

    plt.title(f"Spektrálna analýza – {label} – 29. apríl 2025")
    plt.xlabel("Frekvencia [Hz]")
    plt.ylabel("Amplitúda")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.tight_layout()

    filename = f"{sensor.lower()}_spectrum.png"
    plt.savefig(filename, dpi=300)
    plt.close()

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

# Match UTC interval
match_start = datetime.strptime("2025-05-31 18:30", "%Y-%m-%d %H:%M")
match_end = datetime.strptime("2025-05-31 20:00", "%Y-%m-%d %H:%M")

for sensor in a_sensors:
    df_fft = get_fft_spectra_for_sensor(collection, sensor, "2025-05-31")
    label = sensor_labels[sensor]
    color = sensor_colors[sensor]

    # Split into match and non-match
    df_match = df_fft[(df_fft["time"] >= match_start)
                      & (df_fft["time"] <= match_end)]
    df_other = df_fft[(df_fft["time"] < match_start)
                      | (df_fft["time"] > match_end)]

    plt.figure(figsize=(9, 5))

    # Plot other data in gray
    plt.scatter(df_other["f"], df_other["a"], s=10,
                color="#cccccc", alpha=0.3, label="mimo koncert")
    # Plot match data in sensor color
    plt.scatter(df_match["f"], df_match["a"], s=10,
                color=color, alpha=0.7, label="počas koncertu")

    plt.title(f"Spektrálna analýza – {label} – 31. máj 2025")
    plt.xlabel("Frekvencia [Hz]")
    plt.ylabel("Amplitúda")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.legend(loc="upper right", fontsize=8)
    plt.tight_layout()

    filename = f"{sensor.lower()}_spectrum_highlighted.png"
    plt.savefig(filename, dpi=300)
    plt.close()

In [ ]:
import numpy as np
import matplotlib.colors as mcolors

for sensor in a_sensors:
    df_fft = get_fft_spectra_for_sensor(collection, sensor, "2025-05-31")
    label = sensor_labels[sensor]
    base_color = sensor_colors[sensor]

    # split
    df_match = df_fft[(df_fft["time"] >= match_start)
                      & (df_fft["time"] <= match_end)]
    df_other = df_fft[(df_fft["time"] < match_start)
                      | (df_fft["time"] > match_end)]

    # If there's no match data, skip this sensor
    if df_match.empty:
        print(f"Skipping {label}: no data during match interval.")
        continue

    # prepare alpha gradient
    try:
        amps = df_match["a"].values
        # normalize to [0,1]
        norm_amps = (amps - amps.min()) / (amps.max() - amps.min() + 1e-16)
        # rescale into [min_alpha, max_alpha]
        min_alpha, max_alpha = 0.2, 0.9
        alphas = norm_amps * (max_alpha - min_alpha) + min_alpha

        # build per‐point RGBA
        rgba = np.tile(mcolors.to_rgba(base_color), (len(amps), 1))
        rgba[:, 3] = alphas

    except Exception as e:
        # In case something else goes wrong (e.g. all amplitudes are NaN)
        print(f"Skipping {label} due to error in gradient calculation: {e}")
        continue

    plt.figure(figsize=(9, 5))
    # plot “other” in flat gray
    plt.scatter(df_other["f"], df_other["a"], s=10,
                color="#cccccc", alpha=0.3, label="mimo koncertu")
    # plot “match” with per‐point opacity
    plt.scatter(df_match["f"], df_match["a"], s=10,
                color=rgba, label="počas koncertu")

    plt.title(f"Spektrálna analýza – {label} – 31. máj 2025")
    plt.xlabel("Frekvencia [Hz]")
    plt.ylabel("Amplitúda")
    plt.grid(True, linestyle="--", alpha=0.3)
    plt.legend(loc="upper right", fontsize=8)
    plt.tight_layout()

    filename = f"{sensor.lower()}_spectrum_gradient.png"
    plt.savefig(filename, dpi=300)
    plt.close()